#### Note - This model is actually used for this project and uses the Darknet-19 forward prop outlined in the YOLO_v2 paper.

## YOLO_v2 inspired model

- This model takes in input images of shape (448,448,3) and produces output of shape (14,14,14). In looking at the output of my k-means clustering step in my minibatch data cleaning notebook, I decided to have 2 (1,1,7) encodings such that the model can learn the difference between the dimensions of open and closed palms.

- I will be using stochastic gradient descent with adam optimization and an interatively decreasing learning rate as promoted by the YOLO_v2 model.

- This model is essentially the Darknet-19 model, which I chose over the Darknet-53 resnet model strictly to decrease the time needed for model training.

In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
from keras import backend as K
import matplotlib.pyplot as plt
import latex
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [2]:
# Loading data
X = np.load("../../data/dinorunner/images_448.npy")
y = np.load("../../data/dinorunner/encodings_448.npy")
print(X.shape)
print(y.shape)

(396, 448, 448, 3)
(396, 14, 14, 7)


In [3]:
# Creating testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=1)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(376, 448, 448, 3)
(376, 14, 14, 7)
(20, 448, 448, 3)
(20, 14, 14, 7)


### Tensorflow Placeholders

In [4]:
# Placeholder values for input X,y data
def get_placeholders(x_h,x_w,x_c,y_h,y_w,y_c):
    """
    x_h: Height for x input 
    x_w: Width for x input
    x_c: Channels for x input
    y_h: Height for y input
    y_w: Width for y input
    y_c: Channels for y input
    """
    X = tf.placeholder(tf.float32, name="X", shape=(None,x_h,x_w,x_c))
    y = tf.placeholder(tf.float32, name="y", shape=(None,y_h,y_w,y_c))
    return X,y

In [5]:
# Testing placeholders
tf.reset_default_graph()
with tf.Session() as sess:
    X,y = get_placeholders(448,448,3,14,14,7)
    print("X shape:",X.shape)
    print("y shape:",y.shape)

X shape: (?, 448, 448, 3)
y shape: (?, 14, 14, 7)


### Tensorflow Forward Propagation

In [6]:
# Defining constant layer for 2d convolution, batch norm, and activation
def conv(the_input,layer,f,ks):
    """
    the_input: the layer which will be used as input in conv layer
    layer: specifies the layer number for naming sections of graph
    f (filters): the number of filters to be used for conv layer
    ks (kernel_size): kernel size for conv2d layer
    Note - conv2d layers all use padding
    """
    layer = str(layer)
    Z = tf.layers.conv2d(the_input,filters=f,kernel_size=[ks,ks],strides=(1,1),padding="same",name="Z"+layer,kernel_initializer=tf.contrib.layers.xavier_initializer(seed=0))
    Bn = tf.layers.batch_normalization(Z,name="Bn"+layer)
    A = tf.nn.leaky_relu(Bn,alpha=0.1,name="A"+layer)
    return A

In [7]:
# Building the forward pass based on Darket-19
# Note - forward pass will use leaky_relu
def forward_pass(X):
    input_layer = tf.reshape(X,[-1,448,448,3]) # Input shape of images
    S1 = conv(input_layer,1,32,3)
    P1 = tf.layers.max_pooling2d(S1,pool_size=[2,2],strides=2,padding="valid",name="P1") # 224x224
    S2 = conv(P1,2,64,3)
    P2 = tf.layers.max_pooling2d(S2,pool_size=[2,2],strides=2,padding="valid",name="P2") # 112x112
    S3 = conv(P2,3,128,3)
    S4 = conv(S3,4,64,1)
    S5 = conv(S4,5,128,3)
    P5 = tf.layers.max_pooling2d(S5,pool_size=[2,2],strides=2,padding="valid",name="P5") # 56x56
    S6 = conv(P5,6,256,3)
    S7 = conv(S6,7,128,1)
    S8 = conv(S7,8,256,3)
    P8 = tf.layers.max_pooling2d(S8,pool_size=[2,2],strides=2,padding="valid",name="P8") # 28x28
    S9 = conv(P8,9,512,3)
    S10 = conv(S9,10,256,1)
    S11 = conv(S10,11,512,3)
    S12 = conv(S11,12,256,1)
    S13 = conv(S12,13,512,3)
    P13 = tf.layers.max_pooling2d(S13,pool_size=[2,2],strides=2,padding="valid",name="P13") #14x14
    S14 = conv(P13,14,1024,3)
    S15 = conv(S14,15,512,1)
    S16 = conv(S15,16,1024,3)
    S17 = conv(S16,17,512,1)
    S18 = conv(S17,18,2014,3)
    # Final layer - no batch norm, linear activation
    S19 = tf.layers.conv2d(S18,filters=14,kernel_size=[1,1],strides=(1,1),padding="valid",name="S19",activation=None)
    return S19

In [8]:
# Testing forward prop
tf.reset_default_graph()
with tf.Session() as sess:
    np.random.seed(1)
    X,y = get_placeholders(448,448,3,14,14,7)
    Z19 = forward_pass(X) # Computation graph
    init = tf.global_variables_initializer()
    sess.run(init)
    aZ = sess.run(Z19,feed_dict={X:np.random.randn(1,448,448,3),y:np.random.randn(1,14,14,7)})
    print("Z shape:", str(aZ.shape))

Z shape: (1, 14, 14, 14)


### Tensorflow Cost Function

Using the predicted box [shape(1,1,7)] with the highest IoU score to determine the cost for a given image.

Note - I deviate from the YOLO function function in that for all of the cells that don't hold the actual prediction for the bounding box, I penalize ALL of the confidence predictions (2 per cell in this case). I also penalize the prediction that is not used for the cell that holds the actual prediction. In order to compensate for this change and not greatly increase the cost from just wrong confidence scores I decrease the lambda<sub>noobj</sub> from 0.5 to 0.25. This is due to the fact that I am factoring in 2 times the previous number of confidence scores.

In [9]:
# Returns the sum of all of the confidence scores corresponding w/ cells that don't have prediction
def get_false_conf(Z,mask):
    conf_1 = tf.square(tf.boolean_mask(Z[:,:,:,0:1],mask))
    conf_2 = tf.square(tf.boolean_mask(Z[:,:,:,7:8],mask))
    conf_sum = tf.add(tf.reduce_sum(conf_1),tf.reduce_sum(conf_2))
    return conf_sum

In [10]:
# Returns the values with a specific mask applied to it
def get_box_values(box,mask):
    """
    Index:
    0: confidence there is an object in cell, 1: mid_x, 2: mid_y, 
    3: width, 4: length, 5: prob_open_palm, 6: prob_close_palm
    """
    confidence = tf.boolean_mask(box[:,:,:,0:1],mask)
    mid_x = tf.boolean_mask(box[:,:,:,1:2],mask)
    mid_y = tf.boolean_mask(box[:,:,:,2:3],mask)
    width = tf.boolean_mask(box[:,:,:,3:4],mask)
    height = tf.boolean_mask(box[:,:,:,4:5],mask)
    prob_open = tf.boolean_mask(box[:,:,:,5:6],mask)
    prob_closed = tf.boolean_mask(box[:,:,:,6:7],mask)
    box = {"co":confidence, "mx":mid_x,"my":mid_y,"w":width,"h":height,"d":prob_open,"c":prob_closed}
    return box

In [11]:
# Returns the IoU
def get_iou(box1, box2):
    """
    box1 - coordinates (x1, y1, x2, y2)
    box2 - coordinates (x1, y1, x2, y2)
    """
    xi1 = tf.maximum(box1["x1"], box2["x1"])
    yi1 = tf.maximum(box1["y1"], box2["y1"])
    xi2 = tf.minimum(box1["x2"], box2["x2"])
    yi2 = tf.minimum(box1["y2"], box2["y2"])
    inter_area = tf.maximum((xi2 - xi1),0) * tf.maximum((yi2 - yi1),0) # no neg bounding box

    box1_area = (box1["x2"]-box1["x1"]) * (box1["y2"] - box1["y1"])
    box2_area = (box2["x2"]-box2["x1"]) * (box2["y2"] - box2["y1"])
    union_area = box1_area + box2_area - inter_area
    iou = inter_area / union_area
    
    return tf.reshape(iou,[])

In [12]:
# Returns the (x1,y1),(x2,y2) coordinates for each bounding box as a dict
def get_xy(box_v):
    mid_x = box_v["mx"]
    mid_y = box_v["my"]
    width = box_v["w"]
    height = box_v["h"]
    width = width * 448
    height = height * 448
    mid_x = mid_x * 32 + 224
    mid_y = mid_y * 32 + 224
    
    x1 = mid_x - (1/2*width)
    x2 = mid_x + (1/2*width)
    y1 = mid_y - (1/2*height)
    y2 = mid_y + (1/2*height)
    
    box_xy = {"x1":x1,"x2":x2,"y1":y1,"y2":y2}
    return box_xy

In [13]:
# This returns the information associated with the bounding box prediction with the max IoU
def get_max_iou(box1, box2, y, mask):
    """
    b1_v: bounding box values associated with box1
    b2_v: bounding box values associated with box2
    y_v: bounding box values associated with y
    """
    b1_v = get_box_values(box1,mask)
    b2_v = get_box_values(box2,mask)
    y_v = get_box_values(y,mask)
    # These new coordinates will be used to get the IoU
    b1_xy = get_xy(b1_v)
    b2_xy = get_xy(b2_v)
    y_xy = get_xy(y_v)
    # Getting the Iou for each bounding box prediction
    b1_iou = get_iou(b1_xy, y_xy)
    b2_iou = get_iou(b2_xy, y_xy)
    # Comparing the ious to determine which guess is the ground truth prediction
    def b1(): return b1_v, b2_v
    def b2(): return b2_v, b1_v
    highest_iou_values, lowest_values = tf.cond(tf.less(b1_iou,b2_iou), b2, b1)
    return highest_iou_values, lowest_values

In [14]:
# Stochastic gradient descent so input of shape (1,14,14,14)
# Heavily penalizes negative predictions for height and weight
# Bounding box prediction based on guess with the highest IoU
def cost_function(Z,y,coord=5,noobj=0.25):
    """
    Z - shape (?,14,14,14)
    y - shape (?,14,14,7)
    """
    c_mask_true = y[:,:,:,0:1] > 0 # which cell has the encoding
    c_mask_false = y[:,:,:,0:1] < 1
    
    box1 = Z[:,:,:,0:7] # First guess
    box2 = Z[:,:,:,7:14] # Second guess
    y_v = get_box_values(y,c_mask_true) # values for y for cell with object
    m_v,lowest_values = get_max_iou(box1,box2,y,c_mask_true) # values for highest IoU guess
    false_conf_score = get_false_conf(Z,c_mask_false) # sum of false confidence predictions
    #false_conf_score = false_conf_score + tf.reshape(tf.square(lowest_values["co"]),[])
    
    # penalizing width,length predictions if negative
    m_v["w"] = tf.sqrt(tf.maximum(m_v["w"],0.0))
    m_v["l"] = tf.sqrt(tf.maximum(m_v["h"],0.0))
    
    y_v["w"] = tf.sqrt(y_v["w"])
    y_v["l"] = tf.sqrt(y_v["h"])
    
    # correspond to individual summations of the cost function:
    part1 = coord * tf.reduce_sum(tf.square(y_v["mx"]-m_v["mx"])+tf.square(y_v["my"]-m_v["my"]))
    part2 = coord * tf.reduce_sum(tf.square(y_v["w"]-m_v["w"])+tf.square(y_v["l"]-m_v["l"]))
    part3 = tf.reduce_sum(tf.square(y_v["co"]-m_v["co"]))
    part4 = noobj * false_conf_score
    part5 = tf.reduce_sum(tf.add(tf.square(y_v["d"]-m_v["d"]),tf.square(y_v["c"]-m_v["c"])))
    total_cost = part1 + part2 + part3 + part4 + part5

    return total_cost

In [15]:
# Testing cost function
# predicted cost w/ rounding error is 1.4396
ay = np.zeros((1,14,14,7))
ay[0,0,0,:] = np.array([1,0.5,0.5,0.25,0.25,1,0]) # top left corner
az = np.zeros((1,14,14,14))
az[0,0,0,0:7] = np.array([0.8,0.35,0.35,0.2,0.2,0.8,0.2]) # PRED 1
az[0,0,0,7:] = np.array([0.5,0.4,0.4,0.22,0.22,0.8,0.2]) # PRED 2
az[0,0,1,0] = 1
az[0,1,0,0] = 1
az[0,1,0,7] = 1
az[0,2,0,7] = 1

with tf.Session() as sess:
    y = tf.placeholder(tf.float32,shape=(None,14,14,7))
    Z = tf.placeholder(tf.float32,shape=(None,14,14,14))
    aCost = cost_function(Z,y)
    init = tf.global_variables_initializer()
    sess.run(init)
    tot = sess.run(aCost,feed_dict={Z:az,y:ay})
    print(tot)

1.4395843


### Tensorflow Model

In [18]:
# Building and training YOLO model using stochastic gradient descent
def model(X_train,y_train,lr=0.001,num_epochs=50,print_cost=True):
    tf.reset_default_graph() # resetting graph
    tf.set_random_seed(1)
    seed=0
    costs=[]
    x_h = X_train[0].shape[0]
    x_w = X_train[0].shape[1]
    x_c = X_train[0].shape[2]
    y_h = y_train[0].shape[0]
    y_w = y_train[0].shape[1]
    y_c = y_train[0].shape[2]
    m = X_train.shape[0]
    
    X,y = get_placeholders(x_h,x_w,x_c,y_h,y_w,y_c)
    Z = forward_pass(X)
    cost = cost_function(Z,y)
    optimizer = tf.train.AdamOptimizer(learning_rate=lr).minimize(cost)
    
    init = tf.global_variables_initializer()
    saver = tf.train.Saver() # to save/load model
    with tf.Session() as sess:
        # Loading saved model
        #saver = tf.train.import_meta_graph("../../data/dinorunner/models_stochastic/yolo_model.ckpt.meta")
        #saver.restore(sess, "../../data/dinorunner/models_stochastic/yolo_model.ckpt")
        sess.run(init) # DON'T RUN INIT IF LOADING MODEL
        
        for epoch in range(num_epochs):
            running_cost = 0
            # shuffling training set order each iteration
            seed += 1
            X_train = shuffle(X_train, random_state=seed) 
            y_train = shuffle(y_train, random_state=seed)
            
            for i in range(X_train.shape[0]):
                aX = X_train[i]
                aX.shape = (1,448,448,3)
                aY = y_train[i]
                aY.shape = (1,14,14,7)
                _,temp_cost = sess.run([optimizer,cost], feed_dict={X:aX,y:aY})
                running_cost += temp_cost
                print(running_cost)
                
            costs.append(running_cost)
            if print_cost and epoch % 1 == 0:
                print("Cost at epoch {}: {} <<<<<<<<<".format(epoch+1,running_cost))
                
        loc = saver.save(sess, "../../data/dinorunner/models_stochastic/yolo_model.ckpt")
        return costs

In [17]:
print(X_train.shape)
print(y_train.shape)

(376, 448, 448, 3)
(376, 14, 14, 7)


In [19]:
acosts = model(X_train,y_train,lr=0.0001,num_epochs=20,print_cost=True)

7.570575714111328
13.302615642547607
20.062905311584473
28.75991153717041
36.11800575256348
38.9745876789093
43.07772374153137
46.80519914627075
50.209975719451904
53.27364492416382
56.57485294342041
59.41601777076721
62.790687084198
66.87264776229858
69.32857036590576
73.78550720214844
75.47061836719513
78.21627962589264
82.81124317646027
85.09737765789032
87.7090402841568
90.2771612405777
95.16017305850983
99.54486382007599
102.73099029064178
105.58709514141083
108.01889884471893
110.50891363620758
113.33190023899078
115.9145804643631
118.95929110050201
121.5904620885849
123.64429032802582
125.89225351810455
128.82098019123077
133.6373678445816
135.5106908082962
137.39238107204437
140.0306099653244
143.5664654970169
145.30394279956818
147.6106961965561
149.21590757369995
151.06431221961975
152.71809697151184
154.90798044204712
158.08949065208435
161.53003573417664
164.20301008224487
167.2007508277893
169.04961395263672
171.74506521224976
173.63829708099365
175.5706752538681
178.78785

176.50345134735107
179.05223512649536
180.90638518333435
183.06228303909302
186.445415019989
190.91281509399414
193.1254756450653
195.62647652626038
198.28501105308533
200.37015628814697
202.759827375412
205.12870573997498
207.37103509902954
209.7599925994873
212.6755199432373
216.03994846343994
219.16877269744873
220.79449677467346
222.98141384124756
224.64178919792175
226.5843780040741
228.24654507637024
231.22694849967957
233.17700147628784
236.0824213027954
238.83450603485107
241.1089699268341
243.02054810523987
245.5049421787262
248.76686930656433
250.88724613189697
254.46788144111633
256.3478217124939
259.6701331138611
261.3453493118286
264.1644170284271
266.83795857429504
269.05954813957214
271.1598174571991
273.5243082046509
275.8712124824524
277.30610966682434
280.06865882873535
282.76938343048096
284.489364862442
287.99713468551636
291.0835118293762
293.1965436935425
295.58388566970825
298.2906856536865
301.3976435661316
305.07799768447876
308.07361364364624
309.6743628978729

363.2016267776489
365.2893922328949
367.20185685157776
369.4662125110626
372.1265215873718
374.9765725135803
377.8471140861511
380.0876739025116
381.91929066181183
383.75827896595
386.22228944301605
388.39469015598297
390.73564088344574
393.3921300172806
395.1523530483246
396.7801058292389
398.2479306459427
400.3782788515091
403.13673174381256
405.1402996778488
406.69639027118683
409.39667904376984
412.58333671092987
416.5152200460434
419.0961478948593
421.06195080280304
423.8672305345535
425.94080793857574
428.05890810489655
429.7811952829361
432.1163240671158
434.842715382576
438.32231986522675
439.84444308280945
443.3192105293274
444.892520070076
448.31348621845245
450.98956167697906
453.3628045320511
455.0676453113556
457.1200611591339
459.28740668296814
461.6828532218933
463.2550085783005
465.468443274498
468.11139929294586
471.23691523075104
474.85270273685455
476.54224622249603
478.6251987218857
480.8483589887619
484.68600714206696
486.6587529182434
488.67147159576416
491.209447

534.336635351181
536.3183178901672
538.626704454422
540.6182264089584
543.742485165596
545.8884793519974
547.8500345945358
549.6804473400116
551.4012632369995
553.405978679657
555.2103452682495
557.8785653114319
560.9432375431061
563.7118570804596
566.4546573162079
568.1517783403397
569.8646359443665
572.6381220817566
576.2555375099182
579.738874912262
582.0912353992462
584.0028977394104
585.4754345417023
588.2355697154999
590.1661077737808
592.5932348966599
595.5924197435379
597.2988133430481
599.7481002807617
602.0021677017212
603.8527548313141
605.7408244609833
607.9662718772888
611.5532562732697
613.8993949890137
615.5661268234253
617.0426349639893
618.4970968961716
620.5878232717514
622.3571063280106
624.432976603508
626.7780455350876
628.8821343183517
630.9871393442154
632.9559409618378
635.1035866737366
637.0729744434357
639.317330121994
642.1410973072052
645.3680572509766
647.0109746456146
649.8379819393158
651.7908182144165
654.9039108753204
656.960569858551
660.2833249568939


710.2621296644211
712.3829239606857
714.2571297883987
717.1858111619949
720.1634584665298
723.4098562002182
725.6736377477646
727.8164275884628
731.0145996809006
732.6871727705002
734.9392145872116
737.2510133981705
739.5813266038895
742.1525000333786
744.8153060674667
747.3091558218002
748.8779658079147
751.6684657335281
753.7592562437057
756.4909576177597
759.2143565416336
762.220898270607
764.3791357278824
767.5129851102829
769.804124712944
771.8567794561386
774.3670791387558
775.9393215179443
778.5580015182495
780.428807258606
783.158224105835
785.0312030315399
787.3542454242706
790.4011261463165
792.5558025836945
795.6706097126007
797.3011918067932
799.9581115245819
802.5654566287994
804.7485492229462
806.7487604618073
808.7666628360748
812.4822890758514
814.7111139297485
817.1322932243347
818.7977933883667
820.9055547714233
822.7399821281433
826.4845902919769
828.7179744243622
831.2931067943573
833.3871104717255
835.7070920467377
837.4946441650391
839.4851260185242
841.1129851341

...

152.97560277581215
154.28053119778633
155.28786924481392
157.06451657414436
159.7436366379261
160.8090491592884
162.20888105034828
165.6788560450077
168.06433048844337
170.00395241379738
172.51142850518227
174.11463513970375
175.56658473610878
176.52579978108406
178.61231163144112
179.67564025521278
181.4157419502735
183.3495912849903
185.18086710572243
186.30822983384132
188.64759793877602
190.5313505232334
191.7809752523899
193.33658745884895
194.2381121814251
196.22706684470177
198.35586628317833
199.86103639006615
200.44730696082115
202.13145825266838
203.58456394076347
204.94476339221
206.27986428141594
209.41934844851494
211.88786885142326
213.11303409934044
214.46970781683922
215.71039721369743
217.09852609038353
221.34973153471947
223.28147742152214
224.97464391589165
226.16994699835777
229.05736669898033
229.95050767064095
232.02623036503792
234.36020520329475
236.03554359078407
236.62349697947502
238.06182047724724
239.38340291380882
240.4635302722454
241.6758310496807
243.78

233.649251639843
234.77180808782578
236.3391426205635
237.17454141378403
237.95483201742172
238.33564966917038
241.5312289595604
243.28125590085983
245.1498811841011
247.75616532564163
249.4320769906044
251.24056082963943
252.65856355428696
254.01502627134323
255.9003981947899
258.7177318930626
259.95288068056107
262.5344894528389
263.87782150506973
265.6901186108589
267.13291746377945
268.08679991960526
268.986121237278
270.2676423192024
270.8926982283592
271.77500343322754
274.0541031360626
274.80506098270416
276.32090866565704
277.42673230171204
279.47526264190674
282.87680983543396
283.7896466255188
285.18661868572235
286.7260105609894
288.2167944908142
290.36563086509705
291.68971490859985
292.5327779054642
293.7769786119461
296.65794241428375
298.28367948532104
300.2885866165161
301.0886006951332
302.26390451192856
303.07334500551224
304.2309542298317
305.52257496118546
306.73903769254684
307.9677594304085
309.44714123010635
309.9935572743416
310.4589569568634
311.89866280555725


313.6096348166466
315.47543197870255
316.2982369661331
317.2089239358902
318.0602924823761
319.1740976572037
321.3309370279312
322.44970417022705
323.6879633665085
325.6457725763321
327.3541247844696
330.51330280303955
331.74303901195526
334.1611326932907
335.3877899646759
337.3409034013748
338.90172696113586
340.09357476234436
340.74416023492813
342.5451974272728
343.7905281186104
345.9193187355995
346.8135580420494
347.46081179380417
348.7774813771248
351.8414035439491
353.4079810976982
355.1020364165306
356.55510383844376
359.2673731446266
360.5902619957924
361.9758229851723
364.15705066919327
365.34286254644394
365.76311153173447
366.8022182583809
368.7265024781227
369.15081626176834
369.626337826252
370.9942573904991
373.0168623328209
373.8337759375572
375.56111735105515
376.5394004583359
378.0928683280945
378.9789688587189
380.35764038562775
384.16989743709564
384.76907527446747
385.63347762823105
387.240005671978
389.35341519117355
391.1956755518913
392.26386135816574
392.692214

411.31108662486076
413.91619822382927
415.1696067750454
416.4641636312008
418.6057365834713
419.7075546681881
421.8840146958828
423.10648462176323
423.9856995046139
424.83599469065666
426.1099292933941
426.83189955353737
427.78561213612556
429.4133955538273
430.24904730916023
430.9125235378742
431.5254638493061
433.9303107559681
434.9898444712162
435.8110716044903
436.71186926960945
439.14405539631844
439.76878133416176
440.3320894539356
441.0602205693722
442.0818760097027
444.12647983431816
445.9328133761883
447.3077517449856
448.9288153350353
449.22707962989807
449.7798812389374
450.3629903793335
451.92848765850067
453.19951927661896
456.26907908916473
456.9700580239296
457.512631714344
459.07284992933273
459.9549939632416
461.2282962799072
461.8871220946312
463.10794323682785
464.5843885540962
467.7788673043251
469.0343515276909
469.75510454177856
470.5671426653862
471.4149369597435
472.10229086875916
473.6012783050537
474.3907206058502
475.7577049732208
477.18456768989563
478.26433

465.09305173158646
466.3914403319359
468.063798725605
469.1600312590599
470.2754959464073
471.5726816058159
473.367447912693
477.0285287499428
477.8039398789406
478.6872959136963
Cost at epoch 19: 478.6872959136963 <<<<<<<<<
1.848847508430481
2.8266422748565674
3.9167537689208984
5.10646915435791
6.147822380065918
8.363313913345337
9.974758267402649
11.274402141571045
12.68845546245575
13.372605919837952
13.824057698249817
14.720420897006989
16.677424371242523
17.10533034801483
18.13926100730896
19.79921281337738
20.328867077827454
22.669642567634583
22.8983037173748
23.959903687238693
24.689911514520645
25.576329857110977
27.439275175333023
30.112324625253677
31.49284389615059
32.09636601805687
33.028976291418076
33.906996697187424
34.409697622060776
36.20685288310051
37.35177502036095
39.8877817094326
42.11124262213707
42.59531754255295
44.24321931600571
45.32041209936142
47.154077231884
47.976691126823425
50.674872279167175
53.38722813129425
54.532278656959534
55.98187077045441
56.4

In [34]:
acosts = model(X_train,y_train,lr=0.0003,num_epochs=5,print_cost=True)

7.570575714111328
12.943133354187012
18.22415828704834
22.514138221740723
52.11648082733154
55.33714437484741
60.09353733062744
64.76738595962524
71.28221035003662
75.11364221572876
80.41207885742188
85.69209575653076
90.16326427459717
97.8965196609497
103.11336374282837
107.64448356628418
112.68771934509277
118.32344436645508
126.8720588684082
129.3830442428589
132.99770855903625
136.07670044898987
143.13895058631897
148.57510352134705
152.36800241470337
156.6855206489563
160.5345938205719
163.44989490509033
165.86874508857727
168.76106190681458
171.35277915000916
174.42410445213318
176.5415005683899
179.51275181770325
181.86418676376343
185.4482283592224
187.96805620193481
190.0157823562622
192.96056127548218
196.43495559692383
198.28825461864471
200.59865534305573
202.4245239496231
204.58572113513947
206.66658508777618
208.3215410709381
211.14318680763245
214.34006237983704
216.96646690368652
220.18470311164856
221.87606930732727
224.6506428718567
226.65316700935364
228.321588158607

173.95134937763214
176.4461623430252
179.05029547214508
180.9192122220993
183.10303390026093
186.41436064243317
190.86015713214874
193.07150733470917
195.55831611156464
198.25022041797638
200.3004287481308
202.71619141101837
205.0012012720108
207.25019133090973
209.75086653232574
212.79396069049835
216.19220554828644
219.317023396492
220.88447189331055
223.06406593322754
224.7166200876236
226.6296534538269
228.27980315685272
231.3475819826126
233.32817256450653
236.09544670581818
238.79874217510223
241.1030877828598
243.0175198316574
245.5319868326187
248.7698165178299
250.94142186641693
254.56442821025848
256.4337375164032
259.72629046440125
261.3734256029129
264.16601955890656
266.8850909471512
269.1246875524521
271.26840341091156
273.5682600736618
276.04507076740265
277.4738624095917
280.13572430610657
282.6957983970642
284.47957968711853
288.0019328594208
291.0654489994049
293.1940584182739
295.5573716163635
298.3049054145813
301.4998393058777
305.0759153366089
308.20869636535645
3

362.45489037036896
365.10430586338043
367.20147001743317
369.1209441423416
371.44270408153534
374.087713599205
376.87294471263885
379.8117798566818
382.0002611875534
383.78442561626434
385.66756105422974
388.1672456264496
390.3759162425995
392.8143365383148
395.5159285068512
397.2953083515167
398.9376457929611
400.42296278476715
402.57198894023895
405.3066223859787
407.32711017131805
408.8825491666794
411.5493530035019
414.76863873004913
418.6452616453171
421.2595628499985
423.2454732656479
426.0575760602951
428.11268627643585
430.1991482973099
431.88223564624786
434.17553436756134
436.8213919401169
440.28679406642914
441.75839591026306
445.564820766449
447.0947422981262
450.6912338733673
453.4750974178314
455.7914345264435
457.4890842437744
459.3986117839813
461.75470519065857
464.1596143245697
465.7054486274719
467.8592185974121
470.663507938385
473.80534410476685
477.54193687438965
479.18089842796326
481.17504608631134
483.41293132305145
487.1243802309036
488.8457931280136
490.74765

1158.2494535446167
1164.2886729240417
1170.52818775177
1175.0506324768066
1180.6471991539001
1184.6673035621643
1190.0945110321045
1194.8017029762268
1199.9517178535461
1204.9410948753357
1208.8447511196136
1212.9644577503204
1217.0654094219208
1223.093383550644
1229.7606661319733
1235.233559370041
1240.4408061504364
1244.3062839508057
1248.4173383712769
1254.4389505386353
1261.5243110656738
1269.210765838623
1273.2704434394836
1277.6375727653503
1281.3740055561066
1286.971277475357
1292.290037870407
1297.7332985401154
1303.7260916233063
1307.4652621746063
1313.0666530132294
1318.5216791629791
1323.2269933223724
1328.3945496082306
1333.5501000881195
1339.509513616562
1344.0221889019012
1348.271645307541
1352.0885229110718
1355.90420794487
1361.7782638072968
1366.2393395900726
1371.4918615818024
1376.9751937389374
1382.0157334804535
1386.8483607769012
1391.8142325878143
1396.3166325092316
1400.9770681858063
1405.3238441944122
1411.1235601902008
1417.1564347743988
1421.3825047016144
1426

1421.4138042926788
1425.9739191532135
1430.1113274097443
1434.710393190384
1439.1136186122894
1445.3022754192352
1449.9938380718231
1455.3650901317596
1459.7564942836761
1464.727829694748
1469.4288442134857
1473.4572575092316
1478.1715605258942
1483.439304113388
1489.3577077388763
1493.8170473575592
1498.3557522296906
1504.6604959964752
1508.744467973709
1512.927702665329
1518.36727976799
1523.8392593860626
1528.6000912189484
1534.4215219020844
1540.0212619304657
1544.5597531795502
1549.655350446701
1554.7669894695282
1559.3046305179596
1564.1177823543549
1569.5962026119232
1573.7503774166107
1579.9082877635956
1584.3079216480255
1588.2231063842773
1593.3849229812622
1598.037380695343
1603.2420363426208
1608.0654864311218
1612.579182624817
1617.554648399353
1622.5330700874329
1627.9106125831604
1633.1936955451965
1638.9969396591187
1642.7796874046326
1647.2787232398987
1652.017912387848
1657.3760571479797
1661.2406649589539
1665.3940410614014
1670.8573265075684
1675.9556398391724
1681.

In [36]:
acosts = model(X_train,y_train,lr=0.0001,num_epochs=10,print_cost=True)

INFO:tensorflow:Restoring parameters from ../../data/dinorunner/models_stochastic/yolo_model.ckpt
6.102541923522949
9.982675313949585
15.369159936904907
21.09627890586853
25.84415316581726
30.762438535690308
35.22707962989807
39.73550724983215
44.50891423225403
49.75043559074402
55.017961263656616
59.88081431388855
64.64264178276062
70.02656054496765
74.58385348320007
78.98856234550476
83.88671088218689
89.03323817253113
94.24347424507141
98.98715615272522
103.94185900688171
109.66702246665955
114.81111264228821
120.28316617012024
125.94880700111389
130.34846472740173
134.75917506217957
138.67060542106628
142.63491463661194
147.0427315235138
151.6430423259735
156.86159539222717
161.50757813453674
166.6387369632721
170.58561277389526
176.38846921920776
181.22791576385498
184.8429775238037
190.11954069137573
195.42313194274902
199.37462902069092
203.37558126449585
207.77421283721924
211.48465490341187
215.7166509628296
219.81505727767944
224.17403745651245
229.56366539001465
234.52905750

304.7951033115387
310.2969620227814
315.5565288066864
319.52179503440857
323.89063811302185
328.80738949775696
334.571830034256
340.0924823284149
344.58505940437317
349.89071345329285
355.2542269229889
361.21163296699524
366.0828697681427
371.62076687812805
376.3940246105194
380.4865987300873
386.1072552204132
391.90061020851135
396.8091514110565
402.46761870384216
407.9120695590973
413.041122674942
417.8844907283783
422.8371274471283
427.3428328037262
432.3941924571991
437.56176686286926
442.39349246025085
448.45332407951355
452.81413292884827
457.0798451900482
461.89879870414734
466.3266808986664
470.9691741466522
475.96856713294983
480.4112226963043
485.91620087623596
490.6064670085907
495.6241080760956
499.66325306892395
504.526597738266
510.36447978019714
515.2641456127167
521.1614859104156
525.8413207530975
530.353321313858
535.3813211917877
540.2101762294769
545.0241711139679
549.8202631473541
553.9142668247223
558.062073469162
563.4469139575958
567.9724447727203
573.11662650108

634.9957232475281
639.6376700401306
645.4882054328918
650.6979546546936
654.2658367156982
658.6163487434387
662.721212387085
666.834677696228
671.6297254562378
676.4391345977783
680.3616139888763
683.7784247398376
689.6258506774902
694.1811738014221
699.5740580558777
705.7173581123352
710.804440498352
716.4460835456848
720.7308316230774
725.6562533378601
730.9482235908508
736.3232326507568
742.0640106201172
747.7169299125671
752.5787630081177
757.1737184524536
760.7927565574646
765.7882146835327
770.1680889129639
774.3295764923096
779.130624294281
782.895432472229
786.6089687347412
790.1619162559509
794.54931640625
799.9092378616333
804.7825813293457
808.3213284015656
813.666412115097
819.4516198635101
826.3868625164032
831.8705704212189
835.7416515350342
841.3098835945129
845.9420742988586
851.3686242103577
856.0209136009216
861.311028957367
866.8830189704895
872.549663066864
876.8766050338745
882.0048484802246
886.6092629432678
891.9694499969482
896.7334928512573
902.0369424819946
90

977.2952501773834
981.0593008995056
984.3960726261139
989.7298991680145
993.3403851985931
998.7050082683563
1003.2755205631256
1007.9702923297882
1011.3654148578644
1015.4906165599823
1018.9385492801666
1023.9558041095734
1027.8409118652344
1031.4211654663086
1035.1280462741852
1039.349023103714
1042.869045972824
1046.854644060135
1052.049299955368
1057.8174245357513
1063.745721578598
1068.0694816112518
1073.429206609726
1077.1921775341034
1082.4177033901215
1086.7847774028778
1091.7476260662079
1096.524180173874
1100.2196583747864
1104.1095280647278
1107.9189672470093
1113.6528854370117
1120.1313586235046
1125.4143795967102
1130.5306358337402
1134.012580871582
1137.829351425171
1143.6668162345886
1150.5092978477478
1157.9695000648499
1161.6926896572113
1165.852884054184
1169.4190888404846
1174.8492650985718
1179.9371871948242
1185.226933479309
1190.863401889801
1194.2799487113953
1199.6736946105957
1204.8756880760193
1209.2678866386414
1214.3105688095093
1219.208791732788
1224.9623475

1294.1919965744019
1299.6043639183044
1304.829107761383
1310.1377029418945
1314.79536819458
1319.6707201004028
1324.2214913368225
1329.5849356651306
1333.8942914009094
1338.5394296646118
1343.377851486206
1348.1364464759827
1352.6315693855286
1357.3758583068848
1361.9554595947266
1366.249038696289
1369.7208757400513
1375.0136303901672
1380.7427053451538
1385.1875958442688
1389.2252097129822
1393.7266917228699
1398.0291213989258
1404.0751671791077
1408.684940814972
1413.9555401802063
1418.2722239494324
1423.1637029647827
1427.8200750350952
1431.6788744926453
1436.272373199463
1441.370403289795
1447.158317565918
1451.481701374054
1455.936462879181
1462.1057710647583
1466.1010615825653
1470.1967656612396
1475.5636217594147
1480.9084784984589
1485.5823347568512
1491.3062388896942
1496.8144009113312
1501.2310874462128
1506.234298467636
1511.2763350009918
1515.6957647800446
1520.4001996517181
1525.8078291416168
1529.8528492450714
1535.859011888504
1540.106202840805
1543.9236552715302
1548.93

1608.290804386139
1613.2791619300842
1617.929054260254
1622.4961557388306
1627.1024255752563
1633.5476760864258
1637.9483804702759
1641.9506163597107
1648.4535598754883
1652.6867661476135
1657.8925414085388
1663.2575430870056
1667.8035817146301
1672.21040725708
1677.5506377220154
1681.6704030036926
1686.7631516456604
1691.0759434700012
1696.6515789031982
1701.3343653678894
1706.569905757904
1711.531439781189
1716.149206161499
1720.6093492507935
1725.3364095687866
1730.450255393982
1734.9843153953552
1739.7639718055725
1744.87251663208
1751.381076335907
1755.3750813007355
1760.2592613697052
1764.8641741275787
1769.8485867977142
1775.6509420871735
1779.657279253006
1785.2268402576447
1789.592515707016
1794.1491544246674
1799.1373493671417
1804.9685790538788
1808.71231341362
1812.4608504772186
1816.995986700058
1821.8993499279022
Cost at epoch 6: 1821.8993499279022
4.807959079742432
9.304875373840332
14.116572856903076
18.256322860717773
22.97201681137085
27.413676261901855
33.32575607299

98.83717823028564
102.86203384399414
106.64021182060242
112.33787083625793
117.09579968452454
121.41366791725159
126.85753655433655
131.0790102481842
135.64369940757751
140.79376673698425
146.64000344276428
150.41432547569275
154.65758728981018
159.36139559745789
163.78250241279602
168.27338862419128
172.89193511009216
177.72760033607483
182.48664736747742
188.57158064842224
194.06080222129822
199.5417764186859
203.39794039726257
208.7643644809723
212.99780535697937
218.12961745262146
222.7101218700409
226.31415104866028
230.82608294487
235.88582634925842
243.08843111991882
247.50132632255554
251.65412592887878
256.2199432849884
260.4573781490326
264.95785212516785
269.7433259487152
273.45483922958374
278.51767587661743
284.14305877685547
288.89526987075806
293.7643208503723
299.6021361351013
305.45934438705444
311.0040774345398
314.59376883506775
319.0326769351959
324.496737241745
329.34128737449646
333.6096260547638
337.83490204811096
342.49289774894714
347.1414873600006
351.53679299

430.46422934532166
435.0991451740265
438.9290030002594
442.9433944225311
448.2953631877899
454.246680021286
459.08784794807434
465.41953778266907
468.9451644420624
474.6902849674225
479.74520802497864
485.217223405838
490.22426867485046
493.71351885795593
498.1454360485077
502.13216519355774
505.8494622707367
510.6267726421356
515.1851346492767
520.3989980220795
525.733945608139
529.9298694133759
535.129314661026
539.8392951488495
543.2308549880981
548.1412553787231
552.4108247756958
558.2519345283508
563.1314930915833
568.6106462478638
573.5785140991211
578.2135243415833
582.0884890556335
586.7981338500977
590.5984973907471
596.2652630805969
602.6165175437927
609.0775957107544
613.1623711585999
617.4754509925842
621.8020157814026
626.5792775154114
631.579927444458
635.1695766448975
638.8784878253937
642.5845577716827
647.3000109195709
652.7194283008575
657.4847872257233
662.7482163906097
667.4466068744659
674.1084258556366
678.4026787281036
682.381059885025
688.1543619632721
692.99269

757.2857127189636
761.3342666625977
765.8816151618958
770.4916701316833
774.3590054512024
780.5677938461304
784.6515259742737
790.0664396286011
794.9255719184875
800.0638957023621
805.4892921447754
810.6982364654541
814.6980268955231
819.8896853923798
824.8770859241486
829.4779975414276
834.5230376720428
839.7378966808319
844.3348920345306
849.4112536907196
854.4539268016815
858.7073409557343
864.5798051357269
868.8793694972992
873.518128156662
878.8351304531097
883.7874476909637
888.204220533371
892.0758831501007
896.438873052597
902.3890879154205
907.1149551868439
913.1722614765167
917.2037780284882
921.7877042293549
926.9267318248749
931.6268494129181
935.7927935123444
940.578111410141
947.231847524643
953.1505873203278
957.8688857555389
962.2822482585907
967.6830203533173
972.5485665798187
977.4897773265839
981.1969165802002
984.6646103858948
988.7348022460938
994.4233384132385
998.9469647407532
1004.1149983406067
1009.0888023376465
1013.8584833145142
1019.0209708213806
1026.776849

In [37]:
acosts = model(X_train,y_train,lr=0.0001,num_epochs=10,print_cost=True)

INFO:tensorflow:Restoring parameters from ../../data/dinorunner/models_stochastic/yolo_model.ckpt
6.218000411987305
9.749491930007935
15.192001104354858
21.028122186660767
25.881516218185425
30.596799612045288
35.12692189216614
39.438849687576294
44.352378606796265
49.09535002708435
53.77352786064148
58.21709227561951
62.44061875343323
67.98675560951233
72.21087193489075
77.4934446811676
81.69822573661804
85.93337178230286
91.85662198066711
96.42502760887146
101.66049885749817
106.70188736915588
112.54037165641785
119.19069504737854
125.48479008674622
129.87758469581604
134.1557686328888
137.89575719833374
141.60792207717896
146.10839653015137
149.97127676010132
155.7939805984497
160.57483434677124
165.9687728881836
169.48291325569153
174.6064522266388
179.61665797233582
182.96505737304688
189.29930686950684
193.60574531555176
197.4830183982849
200.98158073425293
205.3964238166809
208.60782504081726
212.40091800689697
216.33608436584473
220.63912439346313
226.4071717262268
230.96770524

296.9968960285187
301.16719937324524
306.5351331233978
311.68186259269714
315.718519449234
319.8500454425812
324.8881356716156
330.7451870441437
336.3201472759247
340.76939940452576
346.144083738327
351.2363154888153
357.61638140678406
362.2921187877655
368.1134240627289
372.98949122428894
377.1167962551117
382.67643427848816
388.618488073349
393.2763235569
398.57330346107483
403.46016001701355
408.90215134620667
414.297970533371
418.8068587779999
422.98954463005066
427.9834988117218
433.5949137210846
439.1560299396515
445.87735247612
450.18474078178406
454.4978058338165
459.36156010627747
463.82533955574036
468.4880769252777
473.8291075229645
477.7767770290375
482.76375555992126
486.6807839870453
491.36632466316223
494.9371235370636
500.83910393714905
505.4151175022125
511.51223397254944
518.6539576053619
523.0461370944977
527.4201853275299
532.2415134906769
536.5855467319489
541.5260255336761
546.4157049655914
550.3608949184418
554.3022494316101
559.7703900337219
564.2502913475037
56

642.5494382381439
647.1157472133636
653.1299011707306
658.3647754192352
661.7581934928894
666.2078738212585
670.3208975791931
674.3109397888184
679.1773347854614
683.8857531547546
687.7774910926819
691.052987575531
696.8160548210144
701.2058787345886
706.7006721496582
712.9024682044983
717.5637192726135
723.476888179779
727.6650981903076
732.6025357246399
738.0413017272949
743.4409141540527
749.0904140472412
754.4197912216187
758.9979195594788
763.3208980560303
767.2575402259827
772.285749912262
776.68048620224
781.0692358016968
785.6908006668091
789.8750338554382
793.6036198139191
797.2997138500214
801.6088664531708
807.2534868717194
812.350460767746
815.7603485584259
820.3395531177521
826.7773287296295
833.5586793422699
839.0093467235565
842.8917801380157
848.2226474285126
852.6285011768341
857.5876543521881
862.231901884079
867.9073932170868
872.7453315258026
878.2280375957489
882.1897649765015
887.9486331939697
892.3237900733948
898.4430708885193
904.0952410697937
909.336588382721


980.1369981765747
987.7174110412598
990.9295105934143
994.2087392807007
999.0943021774292
1003.3382692337036
1010.2751092910767
1015.7357153892517
1021.2884163856506
1024.8324933052063
1029.2318601608276
1032.6471598148346
1037.908370733261
1041.3001821041107
1044.663592338562
1047.9860565662384
1051.7831718921661
1055.6503429412842
1058.985834121704
1064.3903670310974
1071.1325879096985
1077.340853214264
1081.8361058235168
1087.9551424980164
1091.292860507965
1095.9419813156128
1100.164999961853
1105.1465702056885
1110.008252620697
1113.4477033615112
1117.346816778183
1121.2284996509552
1126.497174501419
1132.4437372684479
1137.5754692554474
1142.754357099533
1146.5807554721832
1150.6465637683868
1155.6417796611786
1161.554221868515
1167.9824917316437
1172.4094603061676
1177.399087190628
1181.130616426468
1185.7194607257843
1190.0541174411774
1195.3830058574677
1200.000128030777
1203.9272720813751
1208.8220946788788
1213.5964620113373
1218.1679589748383
1222.5323836803436
1227.3302762

1307.0121002197266
1312.4913039207458
1318.0094847679138
1324.269054889679
1329.6208925247192
1335.48459482193
1340.1912212371826
1344.598424911499
1349.8816766738892
1354.678472518921
1358.510705947876
1363.683448791504
1368.496259689331
1372.9747276306152
1377.0740942955017
1382.9861416816711
1387.1276183128357
1391.9361009597778
1395.324782371521
1400.7309823036194
1406.4684443473816
1411.0768775939941
1415.0704293251038
1419.5236945152283
1423.7099032402039
1428.7802033424377
1433.82887840271
1439.2984809875488
1444.2177233695984
1448.5840005874634
1453.3877959251404
1457.4778728485107
1462.2458109855652
1467.843454837799
1473.575921535492
1477.8779063224792
1482.1987142562866
1488.831630706787
1492.6204915046692
1497.5385074615479
1503.489182472229
1509.1357679367065
1513.7698392868042
1519.4667010307312
1524.643877029419
1529.0561714172363
1534.7789068222046
1539.2849082946777
1543.0211908817291
1546.8289530277252
1551.2423141002655
1554.7912623882294
1563.3485124111176
1568.9028

1625.6867821216583
1632.5044524669647
1637.415734052658
1641.9422643184662
1647.5465610027313
1652.7786509990692
1657.0716273784637
1664.2657945156097
1668.6699540615082
1672.5507383346558
1678.4652962684631
1682.9398159980774
1688.0061464309692
1693.097179889679
1697.8357691764832
1702.8974542617798
1706.953429698944
1711.8421716690063
1716.5190024375916
1720.4109177589417
1727.1341252326965
1730.9501757621765
1736.4131622314453
1741.3928155899048
1747.4447054862976
1750.6530961990356
1755.511920452118
1762.1447834968567
1767.7201089859009
1774.3048481941223
1781.8680324554443
1789.6765608787537
1793.8970947265625
1799.0683665275574
1803.4133305549622
1807.5208830833435
1814.7830204963684
1818.9617052078247
1825.8338074684143
1830.3899750709534
1835.7585353851318
1840.5990343093872
1846.4924445152283
1850.4918246269226
1854.412769317627
1858.8758516311646
1864.8365449905396
Cost at epoch 6: 1864.8365449905396
4.454232215881348
8.803736686706543
12.952849864959717
16.775734424591064
20

88.06298637390137
92.20337247848511
96.32004594802856
100.77191352844238
105.07812118530273
109.00757217407227
115.42872953414917
120.20077085494995
124.36616325378418
130.08319664001465
134.46186113357544
139.3193860054016
144.4352946281433
149.76714944839478
153.19796442985535
158.12162327766418
163.12161564826965
167.29033970832825
172.3024618625641
176.67634081840515
181.7963845729828
186.70878720283508
192.32636284828186
197.31670784950256
202.1043198108673
206.01403784751892
211.50095295906067
215.3774058818817
219.21267676353455
224.48902487754822
228.34493398666382
232.3345901966095
238.0574848651886
247.81828808784485
252.50025486946106
256.40204644203186
260.5528151988983
264.7929193973541
269.7478697299957
275.6856882572174
279.7651264667511
284.4484007358551
288.5911695957184
293.8991768360138
298.8508388996124
305.40027928352356
310.30392718315125
315.707008600235
319.6362302303314
323.7128040790558
329.4391849040985
334.75985741615295
339.17857909202576
343.82757449150085

420.46853256225586
425.7309765815735
429.7588276863098
437.1608762741089
442.4846305847168
445.8296253681183
449.5423381328583
455.19503378868103
461.6292941570282
466.7724039554596
473.4485676288605
476.79786801338196
482.85078978538513
487.89498114585876
493.6786172389984
498.57449650764465
502.0984172821045
506.50510025024414
510.62711000442505
514.6109957695007
519.213002204895
524.2340745925903
529.1003580093384
534.1075296401978
538.6086359024048
543.8719415664673
548.3397078514099
551.9734530448914
556.8460927009583
561.7826905250549
568.2312512397766
572.7903051376343
578.0551919937134
582.8670372962952
587.4046854972839
591.7046751976013
596.0398650169373
600.5118508338928
607.914957523346
614.2713985443115
621.1342959403992
625.5492391586304
629.9067888259888
634.2725048065186
638.5834450721741
644.3154044151306
647.9408535957336
651.8535394668579
655.1176197528839
660.3306744098663
664.5988385677338
670.0160386562347
676.0551908016205
680.7623898983002
688.0871484279633
692.

763.1842119693756
768.229939699173
772.8431594371796
776.6802990436554
780.9288022518158
785.037451505661
789.2505190372467
793.2976777553558
800.9763371944427
805.5687243938446
811.0584485530853
816.0877487659454
820.9131286144257
826.7278211116791
832.3696506023407
837.0135695934296
842.2793438434601
848.5850012302399
852.7401793003082
858.0218408107758
863.8863933086395
868.6347825527191
873.7598383426666
878.3486130237579
882.3893177509308
889.2283070087433
894.6171038150787
899.8726918697357
904.4744017124176
910.1553204059601
915.4203979969025
919.4477736949921
923.9082152843475
930.2666828632355
934.639454126358
940.9865138530731
945.4141490459442
949.2667026519775
953.9244394302368
959.2902474403381
963.1927773952484
968.9225070476532
976.9827449321747
983.1384942531586
988.2020452022552
992.7087113857269
997.2063562870026
1002.2631161212921
1006.9406368732452
1010.7978808879852
1014.4674632549286
1017.759220123291
1023.1547989845276
1028.4443054199219
1034.322211265564
1039.13